In [661]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output


# keras.utils.set_random_seed(253)

In [662]:
ALPHA = 1 # thermal diffusivity coefficient
T_STEP = .1
X_STEP = .2
SOURCE_TEMP = 100
BASE_TEMP = 0
LENGTH = 40
DURATION = 50


def paper_function(x, t)-> float:
    return 6 * np.sin((np.pi * x) / LENGTH) * np.exp(-ALPHA * (np.square(np.pi/LENGTH)) * t)

def paper_initial_function(x)-> float: # t = 0
    return 6 * np.sin((np.pi * x) / LENGTH)


# def initial_function(x: float)-> float: # assumes t = 0
#     if x == 0 or x == LENGTH: # Boundary Conditions, insulated ends
#         return BASE_TEMP
    
#     temp_diff = SOURCE_TEMP - BASE_TEMP
#     if x < .5 * LENGTH:
#         return ((x/LENGTH) * 2 * temp_diff) + BASE_TEMP
#     else:
#         return (temp_diff - (((x - (.5*LENGTH))/LENGTH) * 2 * temp_diff)) + BASE_TEMP
    

# def heat_equation(T_x1, T_x2, T_x3)-> float:
#     # T_x 1, 2, 3 = Temp at x-1, x, and x+1
#     # temps are from previous time step
    
#     return T_x2 + ((ALPHA * T_STEP) * ((T_x3 - (2 * T_x2) + T_x1) / X_STEP**2))

    
# def heat_equation_boundary(t, x)-> float:
#     if x == np.float64(LENGTH) or x == 0: # insulated ends
#         return BASE_TEMP
#     if t == 0:
#         return paper_initial_function(x)

    
    
    


In [663]:
t_set = np.arange(0, DURATION, T_STEP, dtype='float64')
x_set = np.arange(0, LENGTH, X_STEP, dtype='float64')
tx_set = []

# look up
# np.linspace
# np.uniform

x_initial = np.arange(X_STEP, LENGTH, X_STEP, dtype='float64')
t_initial = np.array([0 for item in x_initial], dtype='float64')
x_initial = tf.convert_to_tensor(x_initial, dtype='float64')
t_initial = tf.convert_to_tensor(t_initial, dtype='float64')

u_initial = np.array([paper_initial_function(x) for x in x_initial], dtype='float64')
u_initial = tf.expand_dims(u_initial, axis=1)

# print(u_initial)

# insulated ends
t_boundary = np.array([t for t in t_set] + [t for t in t_set], dtype='float64')
x_boundary = np.array([0 for item in t_boundary[0: int(len(t_boundary) / 2)]] + [LENGTH for item in t_boundary[0: int(len(t_boundary) / 2)]], dtype='float64')


# t_boundary2 = np.arange(0, DURATION, T_STEP, dtype='float64')
# x_boundary2 = np.arange(0, LENGTH + X_STEP, X_STEP, dtype='float64')
# u_boundary = []
# tx_boundary = []

# for item in t_boundary2:
#     for item2 in x_boundary2:
#         if item == 0:
#             tx_boundary.append([item, item2])
#             u_boundary.append(np.array([paper_initial_function(item2)]))
#         elif item2 == 0 or item2 == LENGTH:
#             tx_boundary.append([item, item2])
#             u_boundary.append(np.array([BASE_TEMP]))

# u_boundary = np.array(u_boundary)
# tx_boundary = np.array(tx_boundary)


for item in t_set:
    if item != 0: 
        for item2 in x_set:
            if item2 != 0: 
                tx_set.append(np.array([item, item2]))

tx_set = np.array(tx_set)

# x_b = tf.convert_to_tensor(x_boundary)
# t_boundary = tf.squeeze(t_boundary)
# x_b = tf.squeeze(x_b)

# print(x_b)
# print(t_boundary)




In [664]:
def make_network():
    layers = []         

    layers.append(keras.layers.Dense(20, activation = 'elu', input_shape = (None, 1, 2), dtype='float64'))
    layers.append(keras.layers.Dropout(.5))
    layers.append(keras.layers.Dense(20, activation = 'elu', dtype='float64'))
    layers.append(keras.layers.Dropout(.5))
    layers.append(keras.layers.Dense(20, activation = 'elu', dtype='float64'))
    layers.append(keras.layers.Dropout(.5))
    layers.append(keras.layers.Dense(20, activation = 'elu', dtype='float64'))
    layers.append(keras.layers.Dropout(.5))
    layers.append(keras.layers.Dense(20, activation = 'elu', dtype='float64'))
    layers.append(keras.layers.Dropout(.5))
    layers.append(keras.layers.Dense(20, activation = 'elu', dtype='float64'))
    layers.append(keras.layers.Dropout(.5))
    layers.append(keras.layers.Dense(1, activation = 'linear', dtype='float64'))
    
    network = keras.Sequential(layers)


    return network


In [665]:
def calc_residual(model: keras.Model, tx_list, alpha)-> tf.Tensor:

    t = tx_list[:, 0:1]
    x = tx_list[:, 1:2]
    t = tf.convert_to_tensor(t, dtype='float64')
    x = tf.convert_to_tensor(x, dtype='float64')

    t = tf.squeeze(t)
    x = tf.squeeze(x)

    with tf.GradientTape(persistent= True) as gt:

        gt.watch(t)
        gt.watch(x)

        u = model(tf.stack([t, x], axis=1))
        
        # print("u_x: " + str(du_dx))
        # u = tf.convert_to_tensor(tf.squeeze(u), dtype='float64')

        du_dx = gt.gradient(u, x)

    du_dt = gt.gradient(u, t)
    # print("u_t: " + str(du_dt))
    

    d2u_dx2 = gt.gradient(du_dx, x) # second derivative of predicted values
    # print("u_x2: " + str(d2u_dx2))
    del gt

    return du_dt - (alpha * d2u_dx2)


In [666]:
def calc_loss(model: keras.Model, tx_list, alpha):
    """
    Custom loss function made for neural network. It calculates the residual, and then
    adds it to the difference in the boundary outputs and predicted values.
    Returns the loss.
    
    """

    residual = calc_residual(model, tx_list, alpha)  
    av_residual = tf.reduce_mean(tf.square(residual)) 


    # print(tf.stack([x_initial, t_initial], axis=1))
    # print(tf.stack([tf.squeeze(x_boundary), tf.squeeze(t_boundary)], axis=1))
    

    # at beginning, at t = 0, heat should be a gradient from BASE_TEMP at 0, SOURCE_TEMP in the middle, then back down to BASE_TEMP at the end
    u_i_pred = model(tf.stack([t_initial, x_initial], axis=1))

    u_i_pred = tf.convert_to_tensor(u_i_pred, dtype='float64')

    u_i_diff = u_initial - u_i_pred


    # print(tf.stack([t_initial, x_initial], axis=1))
    # print(u_i_pred)
    # print(u_initial)
    # print(u_i_diff)

    # at any point in time, derivative of x = 0 and x = LENGTH should be 0, as the ends are insulated and heat should not enter or exit
    
    # with tf.GradientTape() as gt:
    
    # x_b = tf.convert_to_tensor(x_boundary, dtype='float64')

    #     gt.watch(x_b)

    u_b_pred = model(tf.stack([tf.squeeze(t_boundary), tf.squeeze(x_boundary)], axis=1))


    # du_dx_pred = gt.gradient(u_b_pred, x_b) # derivative should equal 0, insulated ends should not change temp

    # print(tf.reduce_mean(tf.square(u_i_diff)))

    # print("av residual: " + str(av_residual))
    # print("Initial condition difference: " + str(tf.reduce_mean(tf.square(u_i_diff))))
    # print("Boundary Condition diff: " + str(tf.reduce_mean(tf.square(u_b_pred))))


    return (av_residual) + (tf.reduce_mean(tf.square(u_i_diff))) + (tf.reduce_mean(tf.square(u_b_pred - BASE_TEMP)))

In [667]:
# gradient is of the loss function with respect to all the variables in the model
def calc_gradient(model: keras.Model, tx_list, alpha):
    """
    Calculates the gradient to apply to the neural network. Returns the
    gradient.
    
    """

    with tf.GradientTape() as gt:
        gt.watch(model.trainable_variables) # weights
        loss = calc_loss(model, tx_list, alpha)

    gradient = gt.gradient(loss, model.trainable_variables)

    # print(str(gradient))

    return loss, gradient

In [668]:
def train(model: keras.Model, tx_list, alpha, optimizer: keras.optimizers.Optimizer):
    """
    Executes one training step for the network. Calculates loss and gradient, applies the 
    gradient, then returns the loss.
    
    """
    
    loss, gradient = calc_gradient(model, tx_list, alpha)

    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    return loss

In [669]:
# train the model
epochs = 5000
i = 0
network = make_network()
# network.load_weights("1D_heat_equation_network.h5")

while i < epochs:
    
    loss = 0
    loss = train(network, tx_set, ALPHA, keras.optimizers.Adam())


    print("Epoch: " + str(i))
    print("Loss: " + str(loss))
    print("\n")
    # save every epoch so i can jump ship at any time and still have a network to show for it
    if i % 10 == 0:
        network.save("1D_heat_equation_network.h5")
    i += 1

network.save("1D_heat_equation_network.h5")

Epoch: 0
Loss: tf.Tensor(5.079465891211563, shape=(), dtype=float64)




C:\Users\werea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch: 1
Loss: tf.Tensor(3.9715556323519117, shape=(), dtype=float64)


Epoch: 2
Loss: tf.Tensor(3.1374667097633266, shape=(), dtype=float64)


Epoch: 3
Loss: tf.Tensor(2.5235315522613613, shape=(), dtype=float64)


Epoch: 4
Loss: tf.Tensor(2.0909166106837063, shape=(), dtype=float64)


Epoch: 5
Loss: tf.Tensor(1.7997877478807895, shape=(), dtype=float64)


Epoch: 6
Loss: tf.Tensor(1.6068051352183135, shape=(), dtype=float64)


Epoch: 7
Loss: tf.Tensor(1.441133986940157, shape=(), dtype=float64)


Epoch: 8
Loss: tf.Tensor(1.2974869392362467, shape=(), dtype=float64)


Epoch: 9
Loss: tf.Tensor(1.1888930861665208, shape=(), dtype=float64)


Epoch: 10
Loss: tf.Tensor(1.108492217399481, shape=(), dtype=float64)


Epoch: 11
Loss: tf.Tensor(1.0437472904471854, shape=(), dtype=float64)


Epoch: 12
Loss: tf.Tensor(0.9738264157344775, shape=(), dtype=float64)


Epoch: 13
Loss: tf.Tensor(0.9161108403882686, shape=(), dtype=float64)


Epoch: 14
Loss: tf.Tensor(0.859402702237456, shape=(), dtype=f

KeyboardInterrupt: 

In [670]:
# testing
actual_temps = []

x_full = np.arange(0, LENGTH + X_STEP, X_STEP, dtype='float32')

for number in range(len(t_set)):

    actual_temps.append(np.array([paper_function(x, t_set[number]) for x in x_full]))

    # if not number == 0:
    #     # normally I love list comprehensions, but WOW this is garbage
    #     actual_temps.append(np.array([heat_equation_boundary(t_set[number], x_full[i]) if i == 0 or i == len(x_full) - 1 else heat_equation(actual_temps[number-1][i-1], actual_temps[number-1][i], actual_temps[number-1][i+1]) for i in range(len(x_full))]))

    # else: 
    #     actual_temps.append(np.array([heat_equation_boundary(t_set[number], x) for x in x_full]))
    

actual_temps = tf.convert_to_tensor(actual_temps, dtype='float32')

if True:
    network = make_network()
    network.load_weights("1D_heat_equation_network.h5")
    predictions = []

    x = tf.convert_to_tensor(x_full, dtype='float32')

    for number in t_set:

        time = np.array([number for item in x])

        time = tf.convert_to_tensor(time, dtype='float32')
        
        y = network(tf.stack([time, x], axis=1))
        predictions.append(y)

# return 6 * np.sin((np.pi * x) / LENGTH) * np.exp(-ALPHA * (np.pi/LENGTH)**2 * t)


plt.ion()

# plt.xlabel("Position")
# plt.ylabel("Temp")
# plt.axis((-.2, 6, 0, 101))

# for item in predictions:
#     color = np.where(item < 20, 'k', np.where(item < 40, 'b', np.where(item < 60, 'g', np.where(item < 80, 'y', 'r'))))
#     plt.scatter(x, item, s=5, c=np.squeeze(color), linewidth=0)
    
#     display(plt.scatter(x, item, s=5, c=np.squeeze(color), linewidth=0))
#     clear_output(wait=True)
#     plt.pause(.1)

figure = plt.figure()
sub1 = figure.add_subplot(111)
sub1.set_xlabel("Position")
sub1.set_ylabel("Temp")
sub1.axis((-1, LENGTH + 2, -5, 8))

# line1, = sub1.plot(x, predictions[0], "r--")



for number in range(len(actual_temps)):
   
    color1 = np.where(actual_temps[number] < 1, 'k', np.where(actual_temps[number] < 2, 'b', np.where(actual_temps[number]< 3, 'g', np.where(actual_temps[number] < 4, 'y', 'r'))))    
    sub1.plot(x_full, actual_temps[number], 'b-', linewidth=0.5, zorder=0)
    sub1.scatter(x_full, actual_temps[number], s=5, c=np.squeeze(color1), zorder=1)

    if True:
        color2 = np.where(predictions[number] < 1, 'k', np.where(predictions[number] < 2, 'b', np.where(predictions[number]< 3, 'g', np.where(predictions[number] < 4, 'y', 'r'))))    
        sub1.plot(x_full, predictions[number], 'r--', linewidth=0.5, zorder=2)
        sub1.scatter(x_full, predictions[number], s=5, c=np.squeeze(color2), zorder=3)

    sub1.axis((-1, LENGTH + 2, -5, 8))
    display(figure)
    clear_output(wait=True)
    plt.pause(.1)
    sub1.clear()

    

KeyboardInterrupt: 

In [ ]:

network = make_network()
network.load_weights("1D_heat_equation_network.h5")

t_list = np.array([.6, 2.4, 4.2, 7.7, 9.1, 10.31, 17.9], dtype='float64')
x_list = np.array([.5, 4.83, 10.3, 13.43, 20.7, 25.6, 30.2, 39.9], dtype='float64')

x = tf.convert_to_tensor(x_list)

for item in t_list:
    time = np.array([item for x in x_list])
    t = tf.convert_to_tensor(time)

    with tf.GradientTape(persistent= True) as gt:
        gt.watch(x)
        gt.watch(t)

        u = network(tf.stack([t, x], axis=1))

        u_x = gt.gradient(u, x)
    u_t = gt.gradient(u, t)
    u_xx = gt.gradient(u_x, x)

    del(gt)

    print("\n")
    print("time = " + str(item) + ": ")
    print("u: " + str(u))
    print("uxx: " + str(u_xx))
    print("ut: " + str(u_t))

    




